In [4]:
# Setup environment
import os
import glob
import subprocess
import sys
import time
from scipy.io import netcdf as nc

import matplotlib.pyplot as plt
from os.path import expanduser

# Define output path
home = expanduser("~")
output_dir = os.path.join(home,'scratch/elm_testruns')  # main output directory.
os.makedirs(os.path.dirname(output_dir), exist_ok=True)
print(output_dir)

/Users/sserbin/scratch/elm_testruns


In [17]:
# Grab latest ELM docker container from Docker hub
container_source = "fasstsimulation/elm-builds"
docker_container_tag = "elm_v2-for-ngee"
container_name = ''.join([source,":",docker_container_tag])
print(container_name)
docker_pull_command = ''.join(["docker pull ",source,":",docker_container_tag])
os.system(docker_pull_command)

fasstsimulation/elm-builds:elm_v2-for-ngee
elm_v2-for-ngee: Pulling from fasstsimulation/elm-builds
Digest: sha256:053d765889c59a9d0c3937e36e50f125ad3be83916b6651e450867e845dc341d
Status: Image is up to date for fasstsimulation/elm-builds:elm_v2-for-ngee
docker.io/fasstsimulation/elm-builds:elm_v2-for-ngee


0

In [7]:
# Create docker volumes for output
elmdata_name = "elmdata"
elmoutput_name = "elmoutput"
run_command1 = ''.join(['docker volume create ', elmdata_name])
run_command2 = ''.join(['docker volume create ', elmoutput_name])
#print(run_command)
os.system(run_command1)
os.system(run_command2)

elmdata
elmoutput


0

In [31]:
# Define run options for OLMT
#docker run -t -i --hostname=docker --user modeluser -v elmdata:/inputdata -v elmoutput:/output fasstsimulation/elm-builds:elm_v2-for-ngee /scripts/OLMT_docker_example.sh
hostname="docker"
user="modeluser"
elmdata="elmdata"      # where are the input driver datasets need for ELM?
elmoutput="elmoutput"  # where do you want to put the model output?
#script_to_run="OLMT_docker_example.sh"
script_to_run="OLMT_docker_example_short.sh"

# example input/output locations other than docker volumes
elmoutput="~/scratch/elm_testruns/olmt_test"

elm_run_command = ''.join(['docker run --hostname=', hostname,' --user ', \
                           user, ' -v ', elmdata,':/inputdata -v ', elmoutput, ':/output', \
                           ' ', container_source,':',docker_container_tag, ' /scripts/', \
                          script_to_run])
print(elm_run_command)

docker run --hostname=docker --user modeluser -v elmdata:/inputdata -v ~/scratch/elm_testruns/olmt_test:/output fasstsimulation/elm-builds:elm_v2-for-ngee /scripts/OLMT_docker_example_short.sh


In [49]:
# Run the model simulation
# Create new case
start = time.time()
print("Start date/time: "+time.asctime())
# run the docker container
os.system(elm_run_command)
# get runtime
end = time.time()
print("End date/time: "+time.asctime())
#Print runtime
runtime = (end - start)
print("Total ellapsed time: "+str(runtime)+" seconds")

Start date/time: Tue Aug  9 17:23:31 2022
End date/time: Tue Aug  9 17:23:31 2022
Total ellapsed time: 0.0006909370422363281 seconds


In [ ]:
### Explore output
case_dir='/output/cime_run_dirs/OLMT_AK-K64G_ICB20TRCNPRDCTCBC/run'
ncfiles = ''.join([case_dir,"*.h0.*.nc"])
print(ncfiles)
